# Post-Training an LLM using the Trained Reward Model

This notebook demonstrates how to post-train a Large Language Model (LLM) using our (or any) trained reward model as the objective function. The goal is to align the LLM's outputs to better reasoning trajectories, as judged by a reward model you have already trained (see the [companion notebook](https://github.com/sashapustota/thought-trajectories/blob/main/notebooks/reward_model_training.ipynb) for reward model training).

> **Note:**  
> This notebook is designed for use in **Google Colab** due to the need for GPU acceleration.  
> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1aCr5nkaVAKJDE8HgFZ4W_5XRCvStvhYj?usp=drive_linkg)

## Dependencies
<a name="Dependencies"></a>

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Load our LLM and configure Unsloth
<a name="Model"></a>

Load up `Qwen 2.5 3B Instruct`, the respective tokenizer and set parameters using Unsloth

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-31 09:16:41 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-31 09:16:41 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.43%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness =

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


#### Adding ThoughtMiner class

We are adding our class and unsloth-specific functions manually below as we can't import it due to Google Colab intricacies.

In [ ]:
import torch
import torch.nn as nn
from functools import reduce
import operator
import re
import pickle

from transformers.modeling_utils import PreTrainedModel
from transformers.generation.utils import GenerateDecoderOnlyOutput
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

class ThoughtMiner:

    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizerBase,
    ):
        self.model = model
        self.tokenizer = tokenizer

    def forward_pass_unsloth(
        self, inputs, num_return_sequences: int = 1
    ) -> GenerateDecoderOnlyOutput:
        """
        Performs a forward pass without sampling-based generation.
        Suitable for compatibility with Unsloth training loop.

        Returns:
            GenerateDecoderOnlyOutput: Includes logits and hidden states.
        """
        outputs = self.model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            output_hidden_states=True,
        )
        return outputs

    def get_continuous_thoughts_unsloth(
        self,
        inputs,
        outputs,
        line_delimiter: str = "\n\n",
    ) -> tuple[list, list]:
        """
        Splits output into reasoning steps using double line breaks (paragraphs).
        Only works with model() forward pass, not generate().
        """

        input_ids = inputs["input_ids"]
        batch_size = input_ids.shape[0]

        # Decode each full input (prompt+completion)
        decoded_texts = [self.tokenizer.decode(x, skip_special_tokens=True) for x in input_ids]

        # Last hidden layer (batch, seq, dim)
        hidden_states = outputs.hidden_states[-1]

        continuous_trajectories, token_trajectories = [], []

        for batch_idx in range(batch_size):
            text = decoded_texts[batch_idx]
            tokens = input_ids[batch_idx]

            # Split by two or more consecutive line breaks
            # Handles "\n\n", "\r\n\r\n", or more
            steps = [chunk.strip() for chunk in re.split(r'(?:\r?\n){2,}', text) if chunk.strip()]

            # Tokenize each step
            step_token_ids = [
                self.tokenizer(step, return_tensors="pt", add_special_tokens=False).input_ids[0]
                for step in steps
            ]

            pointer = 0
            traj, tok = [], []
            for step_tokens in step_token_ids:
                length = len(step_tokens)
                # Make sure we do not overrun the sequence
                if pointer + length > tokens.size(0):
                    break
                traj.append(hidden_states[batch_idx, pointer:pointer+length])
                tok.append(tokens[pointer:pointer+length])
                pointer += length

            continuous_trajectories.append(traj)
            token_trajectories.append(tok)

        return continuous_trajectories, token_trajectories

# Initialize the ThoughtMiner module with the model and tokenizer
miner = ThoughtMiner(
    model=model,
    tokenizer=tokenizer
)

### Data Prep
<a name="Data"></a>

In this notebook, we load the **Big-Math-RL-Verified** dataset, using exactly the same data source as in the reward model training notebook. However, to ensure a fair evaluation and prevent data leakage, we **only use questions that were not included in the reward model training set**—specifically, all problems from index 2000 onwards in the dataset.

In [ ]:
# Authenticating with HuggingFace to load the dataset (use your own token)
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `trajectories-testing` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is

In [ ]:
import re
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

# Define prompt
SYSTEM_PROMPT = """
You are MathSolver, an expert in detailed step-by-step mathematical reasoning.

**Instructions:**
1. Chain of thought:
   - Provide your reasoning in short, clear paragraphs.
   - Separate each reasoning paragraph with exactly one blank line.

2. Final answer:
   - After your reasoning, insert exactly one blank line.
   - Then, on a new line, output **only the final numerical answer**. Do **not** include any words, symbols, or explanations. Output just the number.

Problem:
{problem}
"""

def is_numeric_answer(ans):
    clean = ans.strip()
    clean = re.sub(r'^\$|\$$', '', clean)
    return re.fullmatch(r'-?\d+(\.\d+)?', clean) is not None

def get_bigmath_questions(
    split: str = "train",
    system_prompt: str = None,
    start_idx: int = 2000,
    test_size: float = 0.2,
    seed: int = 3407,

) -> Dataset:
    data = load_dataset("SynthLabsAI/Big-Math-RL-Verified", split=split)
    source = "big_math"
    problem_type = "Math Word Problems"

    filtered = [
        ex for ex in data
        if ex.get("source") == source
        and ex.get("domain") and any(problem_type in d for d in ex["domain"])
    ]
    filtered_numeric = [
        ex for ex in filtered
        if ex.get("answer") and is_numeric_answer(ex["answer"])
    ]

    print(f"Found {len(filtered_numeric)} numeric {problem_type} problems from {source}.")

    if system_prompt is None:
        system_prompt = "You are a helpful math tutor. Solve the following math problem step by step."

    examples = []
    for ex in filtered_numeric:
        question = ex["problem"].strip()
        examples.append({
            "prompt": [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": question}
            ],
            "answer": ex["answer"].strip(),
        })

    # Only use questions from index 2000 onward
    examples = examples[start_idx:]
    print(f"Returning {len(examples)} examples (from index {start_idx} onward).")

        # ===== Train/Test Split =====
    train, test = train_test_split(
        examples,
        test_size=test_size,
        random_state=seed,
        shuffle=True
    )
    print(f"Train size: {len(train)}, Test size: {len(test)}")

    train_dataset = Dataset.from_list(train)
    test_dataset = Dataset.from_list(test)

    return train_dataset, test_dataset

train_dataset, test_dataset = get_bigmath_questions(system_prompt=SYSTEM_PROMPT)

README.md:   0%|          | 0.00/6.31k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/32.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251122 [00:00<?, ? examples/s]

Found 3466 numeric Math Word Problems problems from big_math.
Returning 1466 examples (from index 2000 onward).
Train size: 1172, Test size: 294


### Evaluation Function Wrappers
<a name="Evaluation Function Wrappers"></a>

In this section, we define the evaluation functions used to guide post-training of the language model.

We use two types of functions:

1. **Reward Function**  
   We wrap our trained reward model in a function that takes the LLM’s output, splits it into distinct reasoning steps using the `ThoughtMiner` class, applies PCA for dimensionality reduction, and then computes a reward for the entire reasoning trajectory using our trained reward model. This reward is **not** based on the final answer itself, but on the hidden-state dynamics throughout the reasoning process.

2. **Verifier (Control)**  
   As a control, we define a simple verifier that checks if the model’s final answer matches the ground truth, assigning a reward for correct answers. We will use this as a baseline to compare whether our trajectory-based reward function actually improves the model’s reasoning ability.

This setup allows us to test whether aligning the LLM’s internal reasoning dynamics—rather than just the final answer—can improve the quality of its solutions.

#### Reward Function

In [ ]:
import numpy as np
import math

def get_trajectory_reward(
    prompts, completions, answer, reward_model, pca, miner, tokenizer, device="cuda", **kwargs
) -> list[float]:
    rewards = []
    model_device = next(reward_model.parameters()).device

    for i, completion in enumerate(completions):
        full_response = completion[0]["content"]
        q = prompts[i][-1]["content"]

        print("=" * 80)
        print(f"🔹 Prompt: {q}")
        print(f"\n🔹 Full Model Output:\n{full_response}")

        if "<answer>" in full_response:
            reasoning_text = full_response.split("<answer>")[0].strip()
        else:
            reasoning_text = full_response.strip()

        # --- Split reasoning into steps using line breaks ---
        reasoning_lines = [line.strip() for line in reasoning_text.split("\n") if line.strip()]
        if not reasoning_lines:
            print("❌ No reasoning lines found. Skipping.")
            rewards.append(0.0)
            continue

        print("\n🔹 Split Reasoning Steps:")
        for idx, line in enumerate(reasoning_lines):
            print(f"  Step {idx+1}: {line}")

        # --- Reformat with <think> tokens ---
        formatted_reasoning = "\n\n".join(reasoning_lines)
        prompt = f"{q}\n\n{formatted_reasoning}"


        # --- Tokenize and forward ---
        inputs = tokenizer([prompt], return_tensors="pt")

        # Now move to device
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

        with torch.no_grad():
            outputs = miner.forward_pass_unsloth(inputs)

        traj, tokens = miner.get_continuous_thoughts_unsloth(inputs, outputs)

        traj = traj[0]
        tokens = tokens[0]

        if not traj:
            print("❌ No trajectory found after miner.get_continuous_thoughts.")
            rewards.append(0.0)
            continue

        print("\n🔹 Decoded Thinking Steps:")
        for j, step_tokens in enumerate(tokens):
            print(f"  Step {j+1}: {tokenizer.decode(step_tokens)}")

        # --- Step vector prep ---
        max_steps = 5
        step_vecs = [step.mean(0).detach().cpu().float().numpy()
             for step in traj[:max_steps]]

        if not step_vecs:
            print("❌ No usable step vectors extracted.")
            rewards.append(0.0)
            continue

        while len(step_vecs) < max_steps:
            step_vecs.append(np.zeros_like(step_vecs[0]))

        step_matrix = np.stack(step_vecs)
        step_matrix_pca = pca.transform(step_matrix).astype(np.float32)
        step_tensor = torch.tensor(step_matrix_pca).unsqueeze(0).to(device)

        with torch.no_grad():
            reward = reward_model(step_tensor).item()

        print(f"\n✅ Final Reward: {reward:.4f}")
        rewards.append(reward)

    return rewards


#### Verifier

In [ ]:
import re
from typing import Optional

def extract_math_answer(text: str) -> Optional[str]:
    """
    Extracts the final answer from model output.
    Handles \boxed{}, LaTeX blocks, or plain numbers.
    """
    # 1. \boxed{...}
    m = re.search(r'\\boxed\{([^\}]+)\}', text)
    if m:
        return m.group(1).strip()

    # 2. Last non-empty line, clean up
    lines = [l.strip() for l in text.strip().splitlines() if l.strip()]
    if lines:
        last = lines[-1]
        last = re.sub(r'^\$?\$?\\?\[?|\$?\$?\\?\]?$','', last).strip()
        m2 = re.search(r'\\boxed\{([^\}]+)\}', last)
        if m2:
            return m2.group(1).strip()
        # Number pattern
        m3 = re.match(r'^[-\d\.,/^\s]+$', last)
        if m3:
            return last
    # 3. Fallback: any number in last line
    m4 = re.search(r'([-+]?\d*\.?\d+)', lines[-1]) if lines else None
    if m4:
        return m4.group(1)
    return None

def to_number(s):
    """
    Converts string to float (or int), returns None if impossible.
    Handles simple fractions like 3/4 as 0.75.
    """
    if s is None:
        return None
    s = str(s).strip()
    s = re.sub(r'[^\d\.\-\/]', '', s)
    try:
        if '/' in s:
            num, denom = s.split('/', 1)
            return float(num) / float(denom)
        if '.' not in s:
            return int(s)
        return float(s)
    except Exception:
        return None

def get_reward_math(predicted: Optional[str], reference: float, tol=1e-5) -> float:
    """
    Compares model answer to reference, allowing for small tolerance.
    """
    pred_num = to_number(predicted)
    if pred_num is not None and abs(pred_num - reference) < tol:
        return 1.0
    return 0.0

def get_reference_math(example: dict) -> float:
    """
    Returns the reference answer as a float.
    """
    ans = example["answer"]
    ans = re.sub(r'^\$|\$$', '', ans.strip())
    try:
        return float(ans)
    except ValueError:
        m = re.search(r'-?\d+(\.\d+)?', ans)
        if m:
            return float(m.group(0))
        else:
            raise ValueError(f"Could not extract numeric answer from: {ans!r}")

def verifier(prompts, completions, answer, **kwargs) -> list[float]:
    """
    Returns [1.0 if model's answer matches reference numerically, else 0.0].
    """
    # answer is a list of str (ground truth)
    # completions is list of [ {"content": ...} ]
    rewards = []
    for i, completion in enumerate(completions):
        response = completion[0]["content"]
        reference = answer[i]
        # Extract reference value as float
        try:
            reference_num = get_reference_math({"answer": reference})
        except Exception as e:
            print(f"Could not extract reference number for answer: {reference} — {e}")
            rewards.append(0.0)
            continue
        # Extract model prediction as string, then as number
        extracted = extract_math_answer(response)
        r = get_reward_math(extracted, reference_num)
        # Optionally, print debug info:
        print(f"\n{'='*30}\nPrompt: {prompts[i][-1]['content']}\nReference: {reference_num}\nModel output: {response}\nExtracted: {extracted}\nReward: {r}\n")
        rewards.append(r)
    return rewards

#### Load our PCA and Reward Model

Here we upload the pre-trained PCA transformation and reward model files from our local machine to the Colab environment.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reward_model_math_50.pt to reward_model_math_50.pt
Saving pca_math_50.pkl to pca_math_50.pkl


In [ ]:
# Load PCA model from uploaded file
with open("pca_math_50.pkl", "rb") as f:
    pca = pickle.load(f)

input_dim = pca.components_.shape[0]

In [ ]:
# Initialize the reward model and load its weights

device = "cuda"

class RewardLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.cls = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h_last = h_n[-1]
        return self.cls(h_last)

reward_model = RewardLSTM(input_dim=input_dim).to(device)
reward_model.load_state_dict(torch.load("reward_model_math_50.pt"))
reward_model.eval()

RewardLSTM(
  (lstm): LSTM(50, 64, num_layers=2, batch_first=True)
  (cls): Linear(in_features=64, out_features=1, bias=True)
)

### Train the model
<a name="Train"></a>

Now set up GRPO Trainer and all configurations!

In [ ]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 2


#### Train the model with the verifier (control)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        verifier
    ],
    args = training_args,
    train_dataset = train_dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,172 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 119,734,272/3,000,000,000 (3.99% trained)



Prompt: The total in-store price for an appliance is $\textdollar 99.99$. A television commercial advertises the same product for three easy payments of $\textdollar 29.98$ and a one-time shipping and handling charge of $\textdollar 9.98$. Calculate the exact savings in cents when buying the appliance from the television advertiser instead of the in-store price. Provide your answer as a whole number.
Reference: 7.0
Model output: First, let's calculate the total price when buying from the television commercial. The commercial advertises this appliance for three payments of $\textdollar 29.98$ each, plus a one-time shipping and handling charge of $\textdollar 9.98$. We add these amounts together to find the total price from the television:

$$
\text{Total price} = (3 \times \textdollar 29.98) + \textdollar 9.98
$$

To perform this calculation, we first multiply $3 \times \textdollar 29.98$:

\begin{align*}
3 \times 29.98 &= 3 \times (30 - 0.02) \\
&= 3 \times 30 - 3 \times 0.02 \\
&= 90

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / verifier
1,0.000100,1.000000,0.000000,302.500000,0.001360,1.000000
2,0.000000,0.500000,0.707107,231.000000,0.001027,0.500000
3,0.000100,0.000000,0.000000,170.500000,0.002377,0.000000
4,0.000100,0.000000,0.000000,248.500000,0.002210,0.000000
5,0.000000,1.000000,0.000000,492.000000,0.000378,1.000000
6,0.000100,0.000000,0.000000,280.000000,0.002905,0.000000
7,0.000100,0.000000,0.000000,296.000000,0.001269,0.000000
8,0.000000,0.000000,0.000000,906.500000,0.000386,0.000000
9,0.000100,1.000000,0.000000,257.500000,0.001916,1.000000
10,0.000000,1.000000,0.000000,290.500000,0.000835,1.000000


Streaming output truncated to the last 5000 lines.
Let's pair the numbers in the first group \((1, 4, 7, \ldots, 37)\). The first difference is \(4 - 1 = 3\), the second difference is \(7 - 4 = 3\), and so on, up to the last difference \(37 - 34 = 3\). The sum of the differences within this group is \(10 \times 3 = 30\).

Similarly, in the second group \((2, 5, 8, \ldots, 38)\), the first difference is \(5 - 2 = 3\), the second difference is \(8 - 5 = 3\), and so on, up to the last difference \(38 - 35 = 3\). The sum of the differences within this group is also \(10 \times 3 = 30\).

Adding the sums of the differences from both groups, the total sum of all differences is \(30 + 30 = 60\).

**Final answer:** 60
Extracted: 60
Reward: 0.0


Prompt: Twenty pairs of integers are formed using each of the integers \( 1, 2, 3, \ldots, 40 \) once. The positive difference between the integers in each pair is 1 or 3. Find the greatest possible sum of the differences. Express your answer as a sing

TrainOutput(global_step=250, training_loss=3.5735480440052924e-05, metrics={'train_runtime': 3672.4563, 'train_samples_per_second': 0.136, 'train_steps_per_second': 0.068, 'total_flos': 0.0, 'train_loss': 3.5735480440052924e-05})

#### Train the model with our reward function

In [ ]:
from functools import partial, update_wrapper

reward_func = partial(
    get_trajectory_reward,
    reward_model=reward_model,
    pca=pca,
    miner=miner,
    tokenizer=tokenizer,
    # remove device=… here; infer device inside the func
)

# copy over the metadata (including __name__)
update_wrapper(reward_func, get_trajectory_reward)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        reward_func
    ],
    args = training_args,
    train_dataset = train_dataset,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,172 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 119,734,272/3,000,000,000 (3.99% trained)


🔹 Prompt: The total in-store price for an appliance is $\textdollar 99.99$. A television commercial advertises the same product for three easy payments of $\textdollar 29.98$ and a one-time shipping and handling charge of $\textdollar 9.98$. Calculate the exact savings in cents when buying the appliance from the television advertiser instead of the in-store price. Provide your answer as a whole number.

🔹 Full Model Output:
First, let's calculate the total price when buying from the television commercial. The commercial advertises this appliance for three payments of $\textdollar 29.98$ each, plus a one-time shipping and handling charge of $\textdollar 9.98$. We add these amounts together to find the total price from the television commercial.
\[
\text{Total price from TV commercial} = 3 \times 29.98 + 9.98 = 89.94 + 9.98 = 99.92
\]

Next, we consider the in-store price which is $\textdollar 99.99$.

To find the savings, we subtract the in-store price from the total price from the TV c

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / get_trajectory_reward
1,-0.000000,0.112333,0.108475,323.000000,0.000000,0.112333
2,0.000000,0.000000,0.000000,291.500000,0.000000,0.000000
3,0.000000,0.000000,0.000000,268.500000,0.000356,0.000000
4,0.000000,0.012165,0.017203,300.500000,0.000724,0.012165
5,0.000000,0.000000,0.000000,611.500000,0.000225,0.000000
6,0.000000,0.091591,0.061741,367.500000,0.000481,0.091591
7,0.000000,0.107870,0.003115,401.000000,0.000494,0.107870
8,0.000000,0.000000,0.000000,564.000000,0.000258,0.000000
9,0.000000,0.072161,0.009496,297.000000,0.000503,0.072161
10,0.000000,0.000000,0.000000,270.500000,0.000282,0.000000


Streaming output truncated to the last 5000 lines.
  Step 8: \[
  Step 9: 23 \div 5 = 4 \text{ remainder } 3
  Step 10: \]
  Step 11: This means there are 4 complete segments of five terms, and 3 additional terms from the beginning of the next segment.
  Step 12: Next, we calculate the sum of the 4 complete segments:
  Step 13: \[
  Step 14: 4 \times 2 = 8
  Step 15: \]
  Step 16: Now we sum the first 3 terms of the next segment (since 23 terms form 4 complete segments and 3 remaining terms). The next segment is \(4, -3, 2, -1, 0\). We only sum the first 3 terms of this segment:
  Step 17: \[
  Step 18: 4 + (-3) + 2 = 3
  Step 19: \]
  Step 20: Adding the sums of the 4 complete segments and the first 3 terms of the next segment gives us the total sum of the first 23 terms:
  Step 21: \[
  Step 22: 8 + 3 = 11
  Step 23: \]
  Step 24: Therefore, the sum of the first 23 integers is:
  Step 25: **11**

🔹 Reformatted Prompt with <think> tokens:

🔹 Decoded Thinking Steps:
  Step 1: A list of

TrainOutput(global_step=250, training_loss=3.772600616487409e-05, metrics={'train_runtime': 3632.7095, 'train_samples_per_second': 0.138, 'train_steps_per_second': 0.069, 'total_flos': 0.0, 'train_loss': 3.772600616487409e-05})

In [ ]:
# Save our post-trained model with LoRA weights
model.save_lora("grpo_saved_lora")

<a name="Inference"></a>
### Evaluation
We define a function to evaluate the model's performance on the test set using both the verifier and the reward function. This will allow us to compare the effectiveness of the two approaches in guiding the model's reasoning process.

In [ ]:
def evaluate_model_on_math(test_dataset, model, tokenizer, batch_size=4, max_examples=292):
    """
    Evaluates model accuracy on math problems.
    """
    results = []
    num_correct = 0
    num_total = 0

    for idx in range(0, len(test_dataset), batch_size):
        batch = test_dataset[idx: idx + batch_size]
        prompts = batch["prompt"]      # This is now a list of prompts
        references = batch["answer"]   # This is now a list of answers

        # Format prompts for generation
        texts = [tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True) for prompt in prompts]

        # Generate outputs (replace with your vLLM call if needed)
        # For HuggingFace:
        # outputs = model.generate(...)
        # For vLLM:
        from vllm import SamplingParams
        sampling_params = SamplingParams(
            temperature=0.8, top_p=0.95, max_tokens=1024,
        )
        outputs = model.fast_generate(
            texts,
            sampling_params=sampling_params,
            # Change this to None if not using LoRA (for base model)
            lora_request=model.load_lora("grpo_saved_lora"),
        )
        completions = [[{"content": out.outputs[0].text}] for out in outputs]

        print(f"Batch: prompts={len(prompts)}, completions={len(completions)}, answers={len(references)}")

        # Score batch using your existing correctness function
        batch_rewards = math_correctness_reward_func(prompts, completions, references)
        results.extend(batch_rewards)

        num_correct += sum(batch_rewards)
        num_total += len(batch_rewards)

        if max_examples and num_total >= max_examples:
            break

    accuracy = np.mean(results)
    print(f"\nEvaluation accuracy: {accuracy:.2%} ({num_correct}/{num_total})")
    return accuracy, results

#accuracy, results = evaluate_model_on_math(test_dataset, model, tokenizer)

Now we test!

In [ ]:
accuracy, results = evaluate_model_on_math(test_dataset, model, tokenizer)

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Batch: prompts=4, completions=4, answers=4

Prompt: A parabolic arch has a height of 16 inches and a span of 40 inches. Calculate the height of the arch at a point 5 inches from the center. Express your answer in inches.
Reference: 15.0
Model output: The equation of the parabolic arch can be written in the form \(y = ax^2 + bx + c\), where the vertex of the parabola is at the center of the arch. Since the height of the arch at the center (midpoint of the span) is 16 inches, we can assume the vertex is at the origin of a coordinate system, simplifying the equation to \(y = ax^2 + 16\).

The parabola passes through points (20, 0) and (-20, 0) because the span of the arch is 40 inches and the vertex is at the origin. Substituting (20, 0) into the equation, we get:
\[0 = a(20)^2 + 16 \implies 0 = 400a + 16 \implies 400a = -16 \implies a = -\frac{16}{400} = -\frac{1}{25}.\]
Thus, the equation of the parabolic arch is:
\[y = -\frac{1}{25}x^2 + 16.\]

To find the height of the arch at a point

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Batch: prompts=4, completions=4, answers=4

Prompt: Joe had walked half way from home to school when he realized he was late. He ran the rest of the way to school. He ran 3 times as fast as he walked. Joe took 6 minutes to walk half way to school. Calculate the total time it took Joe to get from home to school. Express your answer in minutes as a whole number.
Reference: 8.0
Model output: If Joe walked half the distance to school, it means he ran half the distance to school. Since he took 6 minutes to walk half the distance, it will take him 6 / 3 = 2 minutes to run the remaining distance, as he ran 3 times as fast as he walked.

Let \( t_w \) be the time Joe walked half the distance, and \( t_r \) be the time he ran the other half the distance. We know \( t_w = 6 \) minutes and \( t_r = 2 \) minutes.

The total time \( T \) it took Joe to get from home to school is the sum of the time he walked and the time he ran:
\[ T = t_w + t_r \]
\[ T = 6 + 2 = 8 \] minutes.

**Final answer:**
8


Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Batch: prompts=4, completions=4, answers=4

Prompt: Alec has won 49% of the 200 games of chess he has played. He would like to have won exactly 50% of his games. What is the smallest number of extra games he needs to play to achieve this? Express your answer as a whole number.
Reference: 4.0
Model output: To determine how many additional games Alec needs to play to have won exactly 50% of his games, we start by calculating the number of games Alec has already won and the number of games he needs to win to reach 50%.

First, calculate the number of games Alec has already won:
\[ 49\% \text{ of } 200 = 0.49 \times 200 = 98 \text{ games won} \]

Next, let's denote the number of additional games Alec needs to play as \( x \). The total number of games he will play after playing \( x \) more games will be \( 200 + x \). To have won 50% of his games, the number of games he has won, 98, should be 50% of the total games played. Therefore, we set up the equation:
\[ 98 = 0.5 \times (200 + x) \]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Batch: prompts=4, completions=4, answers=4

Prompt: Person A drove 15 miles at a speed of 30 miles per hour and then increased their speed to 55 miles per hour. How many more miles do they need to drive to achieve an average speed of 50 miles per hour? Express your answer in miles as a whole number.
Reference: 110.0
Model output: First, let's determine the time taken to drive the initial 15 miles at a speed of 30 miles per hour:
- The time taken, \( t_1 \), can be calculated using the formula \( t = \frac{d}{v} \).
- Here, \( d = 15 \) miles and \( v = 30 \) miles per hour, so
\[ t_1 = \frac{15}{30} = 0.5 \text{ hours}. \]

Next, let's find the total distance and total time when the speed is 55 miles per hour. Let \( x \) be the distance driven at 55 miles per hour.
- The total distance driven will be \( 15 + x \) miles.
- The total time taken will be \( 0.5 + \frac{x}{55} \) hours.

We need to achieve an average speed of 50 miles per hour over the total distance. The average speed is 

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Batch: prompts=4, completions=4, answers=4

Prompt: Jitka hiked a trail. After hiking 60% of the length of the trail, she had 8 km left to go. What is the total length of the trail in kilometers?
Reference: 20.0
Model output: Let the total length of the trail be \( x \) kilometers. According to the problem, Jitka hiked 60% of the trail and had 8 km left to go. This means she had already hiked \( 100\% - 60\% = 40\% \) of the trail. We can express the distance hiked as 40% of the total length.

Thus, we can write the equation:
\[ 0.4x = x - 8 \]

Rearrange the equation to isolate \( x \):
\[ 0.4x = x - 8 \]
\[ 0.4x - x = -8 \]
\[ -0.6x = -8 \]

Now, solve for \( x \) by dividing both sides by -0.6:
\[ x = \frac{-8}{-0.6} \]
\[ x = \frac{8}{0.6} \]
\[ x = \frac{80}{6} \]
\[ x = \frac{40}{3} \]
\[ x \approx 13.33 \]

The total length of the trail is:
\[ x = \frac{40}{3} \]

**Final answer:**
\[ \frac{40}{3} \]
Extracted: 40
Reward: 0.0


Prompt: A frog located at (0,0) makes successive ju

Our post-trained model is not much better than the base model, but importantly - it's not worse. If you run the verifier trained model, you will observe that it also does not improve over the base model. This suggests that the problem might be in insufficient training length.